# Housing price one-shot forecasting using a multivariate LSTM model

We want to predict housing prices in the future using the time series data currently available to us. Initially, we attempted to use a simple ARIMA model to forecast future prices, but we realized that such a basic model would not be sufficient for predicting values in an uncertain market like the housing market. Now, we are planning to implement a multivariate Long Short-Term Memory (LSTM) model for future forecasting. The LSTM network is a type of recurrent neural network (RNN) that can be employed to predict values based on time series data. One-shot forecasting means making a prediction for the following month and comparing it to the actual outcome before making the next prediction.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input
import matplotlib.pyplot as plt
from numpy import mean
from sklearn.metrics import mean_squared_error

## 1. Data preperation

In addition to monthly average housing prices that we scraped, we also have the monthly number of housing advertisements on <a href="https://www.kv.ee/">kv.ee</a>, housing loan interest rates over the years, and the average salary over the years.

Firstly, we are going to read and refactor all the data from different sources and merge them into a single time series dataframe.

In [2]:
def readData(district="Tallinn"):
    # 1. Read and prepare housing data
    data = pd.read_csv("../data/lstm/housingData.csv",encoding='latin-1')
    data.date = data.date.astype("str")
    data["date"] = pd.to_datetime(data["date"], format='%m-%Y')
    data = data.replace('-', np.nan)
    data = data.ffill() #Forward Fill 
    data.price = data.price.astype("float")
    data.advertisements = data.advertisements.astype("int")
    data = data.drop(["actives", "city"], axis=1)
    data = data[data.district == district]
    
    # 2. Read and prepare housing loan interest rate data
    dataInterestRate = pd.read_csv("../data/lstm/interestRateData.csv", delimiter=';',  encoding="latin-1")
    dataInterestRate = dataInterestRate.drop(dataInterestRate.columns[0], axis=1)
    dataInterestRate = dataInterestRate.drop(',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,', axis=1)
    dataInterestRate = dataInterestRate.loc[[4]]
    df_melted = pd.melt(dataInterestRate, var_name='date', value_name='interest_rate', ignore_index=False)
    df_melted['interest_rate'] = df_melted['interest_rate'].str.replace(',', '.').str.replace('%', '').astype(float)
    df_melted.reset_index(inplace=True)
    dataInterestRate = dataInterestRate.drop(dataInterestRate.columns[0], axis=1)
    df_melted["date"] = pd.to_datetime(df_melted["date"], format='%m.%Y')
    df_melted = df_melted.drop("index", axis=1)
    
    # 3. Read and prepare income data from first file
    incomeData1 = pd.read_csv("../data/lstm/kuupalk2007-2018.csv", skiprows=2,encoding='latin-1')
    incomeData1['Maakond'] = incomeData1['Maakond'].replace({'..Tallinn': 'Tallinn', 'Tartu maakond': 'Tartu'})
    incomeData1.rename(columns={'Keskmine brutokuupalk, eurot': 'salary'}, inplace=True)
    incomeData1.rename(columns={'Maakond': 'district'}, inplace=True)
    incomeData1.rename(columns={'Aasta': 'year'}, inplace=True)
    incomeData1.rename(columns={'Keskmise brutokuupalga juurdekasvutempo vÃµrreldes eelmise perioodiga, %': 'difference'}, inplace=True)
    incomeData1 = incomeData1[incomeData1.district == district]
    #incomeData1['difference'] = incomeData1['difference'].astype(float)

    kvartal_mapping = {'I kvartal': 1, 'II kvartal': 2, 'III kvartal': 3, 'IV kvartal': 4}
    incomeData1['Kvartal'] = incomeData1['Kvartal'].map(kvartal_mapping)
    incomeData1['year'] = incomeData1['year'].astype(int)

    # Create a new DataFrame with 12 months for each year
    expanded_dates = pd.DataFrame({
        'year': np.repeat(incomeData1['year'].unique(), 12),
        'month': np.tile(np.arange(1, 13), len(incomeData1['year'].unique()))
    })

    # Map each month to its corresponding quarter
    month_to_quarter = {1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 2, 7: 3, 8: 3, 9: 3, 10: 4, 11: 4, 12: 4}
    expanded_dates['Kvartal'] = expanded_dates['month'].map(month_to_quarter)

    # Merge the expanded_dates DataFrame with the original DataFrame
    result_df = pd.merge(expanded_dates, incomeData1, on=['year', 'Kvartal'], how='left')

    # Forward fill the values for 'district', 'salary', and 'difference'
    result_df[['district', 'salary', 'difference']] = result_df[['district', 'salary', 'difference']].ffill()
    result_df['date'] = pd.to_datetime(result_df[['year', 'month']].assign(day=1))
    result_df = result_df.drop(["Kvartal", "year", "month"], axis=1)
    result_df["date"] = pd.to_datetime(result_df["date"], format='%Y-%M')

    # 4. Read and prepare income data from second file
    incomeData2 = pd.read_csv("../data/lstm/kuupalk2018-2022.csv", skiprows=2,encoding='latin-1')
    incomeData2 = incomeData2.drop(index=range(4)) # Starts from 2018, already have these values
    incomeData2['Maakond'] = incomeData2['Maakond'].replace({'Tartu maakond': 'Tartu'})
    incomeData2.rename(columns={'Maakond': 'district','Keskmise brutokuupalga muutus vÃµrreldes eelmise perioodiga, %': 'difference', "Keskmine brutokuupalk, eurot":"salary"}, inplace=True)
    incomeData2.rename(columns={'Maakond': 'district'}, inplace=True)
    incomeData2['difference'] = incomeData2['difference'].replace("..", 6.6)
    incomeData2[['year', 'quartile']] = incomeData2['Vaatlusperiood'].str.split(' ', n=1, expand=True)
    quartile_to_int = {'I kvartal': 1, 'II kvartal': 2, 'III kvartal': 3, 'IV kvartal': 4}
    incomeData2['quartile'] = incomeData2['quartile'].map(quartile_to_int)
    incomeData2['year'] = incomeData2['year'].astype(int)
    incomeData2 = incomeData2.drop("Vaatlusperiood", axis=1)
    incomeData2['difference'] = incomeData2['difference'].astype(float)
    incomeData2 = incomeData2[incomeData2['district'] == district]

    # Create a new DataFrame with 12 months for each year
    expanded_dates = pd.DataFrame({
        'year': np.repeat(incomeData2['year'].unique(), 12),
        'month': np.tile(np.arange(1, 13), len(incomeData2['year'].unique()))
    })

    expanded_dates['quartile'] = expanded_dates['month'].map(month_to_quarter)

    # Merge the expanded_dates DataFrame with the original DataFrame
    result_df2 = pd.merge(expanded_dates, incomeData2, on=['year', 'quartile'], how='left')

    # Forward fill the values for 'district', 'salary', and 'difference'
    result_df2[['district', 'salary', 'difference']] = result_df2[['district', 'salary', 'difference']].ffill()
    result_df2['date'] = pd.to_datetime(result_df2[['year', 'month']].assign(day=1))
    result_df2 = result_df2.drop(["year", "month", "quartile"], axis=1)
    result_df2["date"] = pd.to_datetime(result_df2["date"], format='%Y-%M')

    # Merge both salary dataframes together
    data_salary = pd.merge(result_df, result_df2, on=None, how='outer')
    data_salary = data_salary.sort_values('date')  # Sort the merged DataFrame by the 'date' column
    data_salary = data_salary.drop(["district"], axis=1)
    
    # 5. Merge the datasets
    df_merged = pd.merge(data, df_melted, on='date', how='left')
    df_merged = pd.merge(df_merged, data_salary, on='date', how='left')
    
    return df_merged

In [3]:
data = readData("Tallinn")
data = data.dropna(subset=["salary"]) # Since one time series dataset ended a year earlier, we have to remove those values
data.head()

,date,district,price,advertisements,interest_rate,salary,difference
0,2007-05-01,Tallinn,1993.2,7252,5.63,840.0,8.0
1,2007-06-01,Tallinn,1948.8,7405,5.65,840.0,8.0
2,2007-07-01,Tallinn,1935.0,7644,5.87,799.0,-4.9
3,2007-08-01,Tallinn,1922.1,7945,5.98,799.0,-4.9
4,2007-09-01,Tallinn,1912.4,8476,6.09,799.0,-4.9


## 2. Refactoring data and feature scaling

Next, we are going to refactor the data to a shape that the LSTM needs for input. Firstly, we are normalizing the dataset using min-max scaling. We are doing so because neural networks, including LSTMs, often converge faster when the input features are on a similar scale. Then, we are going to split the data into a training and test set. Each row will have the price as a label, and the values of the previous 60 rows (5 years) will serve as features.

In [ ]:
# Normalize the dataset
scaler_price = MinMaxScaler(feature_range=(0, 1))
scaler_advertisements = MinMaxScaler(feature_range=(0, 1))
scaler_interest_rate = MinMaxScaler(feature_range=(0, 1))

scaled_price = scaler_price.fit_transform(data[['price']])
scaled_advertisements = scaler_advertisements.fit_transform(data[['advertisements']])
scaled_interest_rate = scaler_interest_rate.fit_transform(data[['interest_rate']])

scaled_data = np.concatenate((scaled_price, scaled_advertisements, scaled_interest_rate, data[['difference']]), axis=1)

In [ ]:
# Split the data into training set and test set
train_length = int(len(scaled_data) * 0.8)
val_length = int(len(scaled_data) * 0.1)

train_data = scaled_data[0:train_length,:]
val_data = scaled_data[train_length-60:train_length+val_length,:]
test_data = scaled_data[train_length+val_length-60:,:]


# 1. Create the training data set
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i,:]) # For every row take the values for past 60 rows
    y_train.append(train_data[i,0]) # Actual price 
    
# Convert the x_train and y_train to numpy arrays and reshaping
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 4))



# 2. Create the validation data set
x_val = []
y_val = data.iloc[train_length:train_length+val_length, :]['price']

for i in range(60, len(val_data)):
    x_val.append(val_data[i-60:i, :])

# Convert the data to a numpy array
x_val = np.array(x_val)



# 3. Create the testing data set
x_test = []
y_test = data.iloc[train_length+val_length:, :]['price']

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, :])

# Convert the data to a numpy array
x_test = np.array(x_test)


## 3. Defining model arcitecture

We are going to specify the model architecture that will be used going forward. The architecture includes an LSTM layer, followed by densely connected layers. The model is trained using mean squared error (mse) loss and the Adam optimizer. All other values, such as the node count on each layer and data used for model fitting, can be passed into the function as parameters.

In [ ]:
def create_lstm_model(x_train, y_train, config): 
    layer1_nodes,layer2_nodes,layer3_nodes, n_batch, n_epochs = config
    model = Sequential()
    model.add(Input(shape=(x_train.shape[1], 4)))
    model.add(LSTM(layer1_nodes, activation='relu', return_sequences=False))
    model.add(Dense(layer2_nodes))
    model.add(Dense(layer3_nodes))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    model.fit(x_train, y_train, epochs=n_epochs, batch_size=n_batch, verbose=0)
    return model

## 4. Using grid search to find the best hyperparameters

To determine the optimal hyperparameters for our model, we opted to employ grid search. Initially, our intention was to use <code>GridSearchCV</code> from the scikit-learn library. However, after encountering numerous errors using scikeras wrappers for the Tensorflow model and finding limited helpful information online, we decided to resort to manual implementation.

**The function callout for grid search is commented out** since its execution time is prolonged. The first grid search we conducted lasted for 18 hours.

In [ ]:
def model_configs(): # Define the hyperparameter grid
 layer1_nodes = [50, 75, 100]
 layer2_nodes = [25, 50, 75, 100]
 layer3_nodes = [25, 50, 75]
 n_batch = [32, 64]
 n_epochs = [400, 700, 1000, 1400]
 
 configs = list()
 for i in layer1_nodes:
    for j in layer2_nodes:
         for k in layer3_nodes:
            for m in n_batch:
                for o in n_epochs:
                    configs.append([i, j, k, m, o])
 print('Total configs: %d' % len(configs))
 return configs

In [ ]:
def evaluate(x_train, y_train, x_val, y_val, config):
    model = create_lstm_model(x_train, y_train, config)
    predictions = model.predict(x_val)
    predictions = scaler_price.inverse_transform(predictions) # Undo scaling
    rmse = np.sqrt(mean_squared_error(y_val, predictions))
    return rmse

In [ ]:
def repeat_evaluate(x_train, y_train, x_val, y_val, config, n_repeats=15):
    evaluation_scores = []
    
    # fit and evaluate the model n times
    for _ in range(n_repeats):
        validation_score = evaluate(x_train, y_train, x_val, y_val, config)
        evaluation_scores.append(validation_score)
        
    key = str(config)
    print("Evaluation scores: ", evaluation_scores)
    result = mean(evaluation_scores)
    print('> Model[%s] received RMSE %.3f' % (key, result))
    return result

In [ ]:
def grid_search(x_train, y_train, x_val, y_val, configurations):
    
    # Evaluate configurations and store results in a list
    evaluated_configs = []
    for config in configurations:
        evaluation_result = repeat_evaluate(x_train, y_train, x_val, y_val, config, 20)
        evaluated_configs.append((config, evaluation_result)) # store the configuration and repeated evaluation score as a pair 

    # Sort configurations by error in ascending order
    sorted_configs = sorted(evaluated_configs, key=lambda x: x[1])
    return sorted_configs

In [ ]:
configs = model_configs()
scores = []

#%time scores = grid_search(x_train, y_train,x_val, y_val, configs)

# Find the three configurations with the best error values
for configuration, error in scores[:3]:
 print(configuration, error)


## 5. Building the final model and making predictions

We are going to fit the final LSTM model with hyperparameters we found using grid search in the previous step. For fitting we are going to use all the available training + validation data. After the model has been trained, we will predict values for the testing data and calculate the RMSE.

In [ ]:
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Build the LSTM model
model = Sequential()
model.add(Input(shape=(x_train.shape[1], 4)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(75))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=1000)

# Get the models predicted price values.
# We are combining and predicting values for both the validation and test datasets for better visualization on the plot

# Concatenate x_val and x_test
x_combined = np.concatenate((x_val, x_test), axis=0)
predictions = model.predict(x_combined)
predictions = scaler_price.inverse_transform(predictions) # Undo scaling

# Calculate RMSE
y_combined = np.concatenate((y_val, y_test), axis=0)
rmse = np.sqrt(mean_squared_error(y_combined, predictions))
print('Root Mean Square Error:', rmse, " €")


## 6. Plotting the results

Lastly, we are plotting the one-shot forecasting predictions of the model together with the actual values to gain a visual understanding of how well the model is performing. To scale the plot appropriately and display predictions on a larger scale, we are only plotting the training data starting from the year 2013.

In [ ]:
train = data[72:train_length]
test = data[train_length:].copy()
test.loc[72:, 'Predictions'] = predictions
plt.figure(figsize=(8,5))
plt.ylabel('Average Housing Price')
plt.plot(train["date"], train['price'])
plt.plot(test["date"], test[['price', 'Predictions']])
plt.legend(['Training', 'Test', 'Predictions'])
plt.show()